<a href="https://colab.research.google.com/github/VellummyilumVinoth/Aspect_based_sentimental_analysis/blob/main/recommendation_system_for_old_and_new_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# Step 1: Load the dataset
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/zidan/aspect_extracted_file_info.csv')

In [32]:
dataset

,Product Name,Category,Comment,Overall Sentiment Score,reviewer_name,age,gender,area
0,"Sunlight Care Detergent Liquid, 1L",Laundry,I purchased 2 bottles. Both seem partially use...,0.208559,Wilson Matthew,69,Female,Northern
1,"Sunlight Care Detergent Liquid, 1L",Laundry,Very fast & safe delivery . Neet packing. I wi...,0.505488,Wilson David,27,Male,North Central
2,"Sunlight Care Detergent Liquid, 1L",Laundry,Fast delivery during this pandemic. Got the pr...,0.747320,Christopher Jane,25,Female,North Central
3,"Sunlight Care Detergent Liquid, 1L",Laundry,Alot of liquid has leaked.im very much disappo...,0.000257,Smith Olivia,74,Female,North Central
4,"Sunlight Care Detergent Liquid, 1L",Laundry,Thank you very much !! I received my package t...,0.599767,Brown David,63,Female,Eastern
...,...,...,...,...,...,...,...,...
1774,Puredale Full Cream Milk Powder 400g,MilkPowder,very good product value for the price,0.999854,Wilson Albert,20,Female,Northern
1775,Puredale Full Cream Milk Powder 400g,MilkPowder,Delivery within 1day to Tangalle... well packe...,0.993837,Kumar Saini,42,Male,Southern
1776,Puredale Full Cream Milk Powder 400g,MilkPowder,high quality reasonable price,0.999595,Bathri Vinoth,26,Male,Western
1777,Puredale Full Cream Milk Powder 400g,MilkPowder,Excellent packaging. And it was delivered quic...,0.494646,Soosai Vinoth,52,Male,North Central


## Content based filtering for new user using age, gender and area

In [33]:
# Get the top 10 products in each category from the 'top_products_df'
top_products_df = pd.read_csv('/content/drive/MyDrive/zidan/top_10_products_per_category.csv')


In [34]:
top_products_df

,Rank,Product Name,Category,Overall Sentiment Score
0,1,Herb Line Fat Burner Green Tea 250g,Tea,0.787239
1,2,Zesta - Premium Tea 195g - Pouch,Tea,0.786446
2,3,Green Tea 25 Tea Bags Carton,Tea,0.572055
3,4,"Laojee Pure Ceylon Black Tea Pouch, 200g",Tea,0.530651
4,5,Cardomom Tea 25 Tea Bags Carton,Tea,0.489409
...,...,...,...,...
120,6,Pink Pie Strawberry Flavour Mini Snack Bisuit ...,Biscuit,0.500681
121,7,Cocomo Chocolate Filled Biscuits Bisconni Crea...,Biscuit,0.491105
122,8,Munchee Milk Short Cake Biscuits 200G,Biscuit,0.462778
123,9,Chocolate Puff 200g,Biscuit,0.423929


In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

reviews = dataset["Comment"]
product_names = dataset["Product Name"]
overall_sentiment_score = dataset["Overall Sentiment Score"]
reviewer_name = dataset["reviewer_name"]
age = dataset["age"]
gender = dataset["gender"]
area = dataset["area"]

# Combine relevant features into a single DataFrame
features = pd.DataFrame({'Comment': reviews, 'Product Name': product_names, 'Age': age, 'Gender': gender, 'Area': area})

# Create a TF-IDF vectorizer to convert text into numerical features
tfidf = TfidfVectorizer(stop_words='english')

# Apply TF-IDF vectorization to the 'Review' column
review_matrix = tfidf.fit_transform(features['Comment'])

# Compute the cosine similarity matrix between review texts
cosine_similarities = linear_kernel(review_matrix, review_matrix)

# Define a function to get top recommendations based on demographic similarity
def get_recommendations(user_age, user_gender, user_area, top_n=10):
    # Find the indices of users with similar demographics
    similar_users = features[(features['Age'] == user_age) & (features['Gender'] == user_gender) & (features['Area'] == user_area)].index

    # Calculate the weighted average of cosine similarities with similar users
    weighted_similarities = cosine_similarities[similar_users].mean(axis=0)

    # Sort the indices based on weighted similarity scores
    top_indices = weighted_similarities.argsort()[::-1][:top_n]

    # Get the unique product names of the top recommendations
    top_recommendations = product_names[top_indices].unique()

    return top_recommendations

# Example usages:
# user_age = 68
# user_gender = 'Female'
# user_area = 'Northern'

# user_age = 26
# user_gender = 'Male'
# user_area = 'Western'

user_age = 32
user_gender = 'Female'
user_area = 'North Central'

unique_products = get_recommendations(user_age, user_gender, user_area, top_n=10)

# Recommend Rank 1 product for each category from the dataset
category_recommendations = []
for category in top_products_df['Category'].unique():
    category_products = top_products_df[top_products_df['Category'] == category]
    rank1_product = category_products.loc[category_products['Rank'] == 1, 'Product Name'].iloc[0]
    category_recommendations.append(rank1_product)

# Convert unique_products to a Python list
unique_products = unique_products.tolist()

# Fill remaining slots with category recommendations if needed
if len(unique_products) < 10:
    remaining_slots = 10 - len(unique_products)
    category_recommendations = category_recommendations[:remaining_slots]
    unique_products.extend(category_recommendations)

# Print the recommendations
for i, product in enumerate(unique_products, 1):
    print(f"Rank {i}: {product}")


Rank 1: Toren Dark Chocolate 55g
Rank 2: Zesta - Premium Tea 195g - Pouch
Rank 3: Diva Rose Fresh Laundry Detergent Soap 115G
Rank 4: Chinese Noodles 500g
Rank 5: Dil Maldive Fish Pieces 100g
Rank 6: Fresh Peanuts - 1Kg
Rank 7: Wijaya Jaffna Curry Powder 100g.
Rank 8: Nipuna Samba Rice - 10kg
Rank 9: Samyang Spicy & Cheese Ramen Bowl 105g
Rank 10: Fortune Coconut Oil 01 Ltr


## Collaborative filtering for old user using their sentimental score for the reviews by them

In [36]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Filter the data for the old user
old_user = 'Brown David'
old_user_data = dataset[dataset['reviewer_name'] == old_user]

# Filter the data for products with sentiment score less than 2
filtered_data = dataset[dataset['Overall Sentiment Score'] > 0.999].copy()

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Apply TF-IDF vectorization to the review text
review_vectors = vectorizer.fit_transform(filtered_data['Comment'])

# Calculate cosine similarity between the old user's reviews and the filtered products
cosine_similarities = cosine_similarity(vectorizer.transform(old_user_data['Comment']), review_vectors)

# Get the average sentiment score for each product
filtered_data['Average_Sentiment'] = filtered_data.groupby('Product Name')['Overall Sentiment Score'].transform('mean')

# Combine the average sentiment scores with the cosine similarities
filtered_data['Score'] = cosine_similarities[0] * filtered_data['Average_Sentiment']

# Sort the data based on the score
sorted_data = filtered_data.sort_values('Score', ascending=False)

# Add ranking numbers to the sorted data
sorted_data['Rank'] = range(1, len(sorted_data) + 1)

# Get the top 10 recommended products with ranking numbers
top_10_products = sorted_data[['Rank', 'Product Name', 'Score']].head(10)

print("Top 10 recommended products for", old_user, "with their past reviews:")
print(top_10_products)


Top 10 recommended products for Brown David with their past reviews:
      Rank                                       Product Name     Score
1571     1                    Toscana Ceylon Coffee beans 1Kg  0.434594
1509     2                   Araliya Premium Samba Rice - 5Kg  0.393830
1648     3  Nuts & Co Californian Almonds Nuts 200G (Premium)  0.380407
515      4                        Wijaya White Rice Flour 1Kg  0.350650
1545     5  Premium Quality And Healthy Lavazza Qualita 10...  0.346440
946      6                         Munchee Cream Cracker 490G  0.346162
1090     7                              Cassava (Tomato) 50 g  0.331262
928      8              Munchee Milk Short Cake Biscuits 200G  0.320756
637      9                       Dil Maldive Fish Pieces 100g  0.316997
1155    10                         1pc Gulab Jamun Mix - 160g  0.303873
